# OpenStreetMap Data Case Study

Map Area: Berlin, Deutschland
* https://www.openstreetmap.org/node/240109189  

* https://mapzen.com/data/metro-extracts/metro/berlin_germany/  

    
This is the map of my current city of residence. I've only been living here a short while and thought this would be a good opportunity to analyze the city map in a unique way

## Process
The full extract from mapzen is 2.4 gigs. Using Udacity's provided code I created a subset of every 10th node for a much smaller and manageable sample file. 

From this sample file I ascertained a few interesting details with the data:

* The postal codes format did not use a standard tag. Also postal code data may be in list format for some fields. There was also present postal codes that were not from the Berlin designated postal code  

* Country values were not standard.  

* There were many nodes with 'simc' tags that refer to the Polish simc city location value. 

* There was some abbreviations in use but sparingly. However, a street name may or may not have a conjoined  name in German streets. For example 'Plettenberger Pfad' should not be named 'Plettenbergerpfad', and 'Bergrutenpfad' should not be named 'Bergruten Pfad'. Initially I thought to split these street examples into a tuple (street, street:type for example), but decided this could generate confusion with the locals, where the names should not be split!



### Postal Codes
I found that there were multiple tags describing postal codes including postcode, postal_code, addr:postcode. I elected to standardize the tag to postcode. Also, one tag postal_codes was often a list. I created a function that would extra the list data and create multiple postcode tags for a node. The nodes that had this were typically describing suburbs of Berlin. 

Lastly, the only Berlin valid postal codes should be between 10115 and 14199. There were many node points outside of this, including many Polish codes. These nodes were removed from the data set.

Below are queries used to show most tagged postal codes, suburbs and a correlation of the two. Only the top 20 were shown, with counts higher than 1000.

          select count(postal.id), postal.value, suburbs.value from 
              (select * from nodes_tags union all select * from ways_tags) as postal, 
              (select * from nodes_tags union all select * from ways_tags) as suburbs,
          where postal.[key] like 'postcode' 
              and suburbs.[key] = 'suburb' 
              and suburbs.id = postal.id 
          group by postal.value, suburbs.value 
          having count(postal.id) > 1000 
          order by count(postal.id) desc
          limit 20

          select count(postal.id), suburbs.value from 
              (select * from nodes_tags union all select * from ways_tags) as postal, 
              (select * from nodes_tags union all select * from ways_tags) as suburbs 
          where postal.[key] like 'postcode' 
              and suburbs.[key] = 'suburb' 
              and suburbs.id = postal.id 
          group by suburbs.value 
          having count(postal.id) > 1000 
          order by count(postal.id) desc 
          limit 20 
          
<table>
<tr>
<td>Tagged Postal Codes with Suburb</td>
<td>Suburb Count</td>
</tr>
<tr>
<td><pre><code>
  Count | Pcode | Suburb
  11410 | 12623 | Mahlsdorf
   8002 | 12683 | Biesdorf
   6742 | 12621 | Kaulsdorf
   6363 | 12355 | Rudow
   6218 | 13591 | Staaken
   5972 | 13503 | Heiligensee
   5843 | 12524 | Altglienicke
   5669 | 14089 | Kladow
   5268 | 13465 | Frohnau
   5115 | 13125 | Karow
   5055 | 13127 | Französisch Buchholz
   4707 | 13467 | Hermsdorf
   4382 | 12305 | Lichtenrade
   4206 | 12555 | Köpenick
   4115 | 12526 | Bohnsdorf
   4003 | 13156 | Niederschönhausen
   3985 | 12357 | Rudow
   3949 | 12589 | Rahnsdorf
   3844 | 10318 | Karlshorst
   3699 | 13129 | Blankenburg
</code></pre></td>
<td><pre><code>
  Count | Suburb
  12786 | Lichterfelde
  11708 | Zehlendorf
  11440 | Mahlsdorf
  10497 | Rudow
  10260 | Lichtenrade
   8532 | Köpenick
   8405 | Prenzlauer Berg
   8002 | Biesdorf
   7509 | Mitte
   7384 | Staaken
   7299 | Kaulsdorf
   6962 | Charlottenburg
   6517 | Kreuzberg
   6324 | Buckow
   6279 | Schöneberg
   6185 | Neukölln
   6131 | Mariendorf
   6102 | Heiligensee
   5888 | Altglienicke
   5873 | Friedrichshain
</code></pre></td>
</tr>
</table>

The highest count for an individual postal code came from Mahlsdorf, a suburb of Marzahn-Helledorf. This area is on the far eastern end of the city and one postal code covers the entirety of the suburb. Of the remaining top 20 the 19 are prefixed 12/13 and 14 which denotes the larger sections outside the city's Mitte.

Here is a map of the cities postal code areas.
https://upload.wikimedia.org/wikipedia/commons/5/56/Berlin_Postleitzahlen.svg

Comparing the first table to the second, we see representation from the more popular Mitte neighbourhoods in Berlin (Prenzlauer Berg, Mitte, Friedrichshain . I found it interesting to see Lichtenhelde at the top, and dared to look deeper at this result and compare to the 2nd most of Zehlendorf. Lichtenhelde has 5 postal codes according to the wiki on the suburb, and Zehlendorf only 4. The result set below shows 11 in the dataset for Lictenhelde and 5 in the dataset for Zehlendorf.
https://de.wikipedia.org/wiki/Berlin-Zehlendorf

https://de.wikipedia.org/wiki/Berlin-Lichterfelde

          select distinct postal.value, suburbs.value from 
          (select * from nodes_tags union all select * from ways_tags) as postal, 
          (select * from nodes_tags union all select * from ways_tags) as suburbs 
          where postal.[key] like 'postcode' 
          and suburbs.[key] = 'suburb' 
          and suburbs.id = postal.id 
          and suburbs.[value] = 'Lichterfelde' #'Zehlendorf'
          order by postal.value asc 
         
<table>
<tr>
<td>Counts for Lichterfelde</td>
<td>Counts for Zehlendorf</td>
</tr>
<tr>
<td><pre><code>
  Count | Postal Code
   3147 | 12209
   2873 | 12205
   2710 | 12207
   2198 | 12203
   1695 | 14167
    111 | 14195
     21 | 12247
     13 | 12165
     12 | 12279
      6 | 14169
</code></pre></td>
<td><pre><code>
  Count | Postal Code
   3270 | 14163
   3081 | 14169
   2716 | 14165
   1879 | 14167
    762 | 14129
    
    
    
    
    
</code></pre></td>
</tr>
</table>

The results for Lichterfelde show the top 5 results belong to the correct suburb. 14167 is valiud and belongs to both. 14195 howeverand below are invalids for Lichterfelde and used by other neighbouring suburbs. For Zehlendorf 14129 also belongs to another suburb. These data points are certainly tagged incorrectly.


### Country

Many country tag's simply used DE. However there were several with Deutschland or Germany. These were simply converted to DE. 

I also found many other country tags in the listing that me scratching my head for a moment. One interesting finding was that since Berlin is the capital, there are many embassies on the map using a different country code. According to Berlin.de there is 151 different embassy buildings. https://www.berlin.de/tourismus/adressen/botschaft/
   
     select distinct id 
     from (select * from nodes_tags union select * from ways_tags) 
     where key = 'amenity' and value = 'embassy'"
    
     select distinct [value]
     from (select * from nodes_tags union select * from ways_tags)
     where id in (%s) and key in ('name') order by [value]"

      # %s is the id's for the first query
<table>
<tr>
<td><pre><code>
Angolanische Botschaft
Apostolische Nuntiatur in Deutschland
Australische Botschaft
Belgische Botschaft
Bevollmächtigte der Freien Hansestadt Bremenbeim Bund, für Europa und Entwicklungszusammenarbeit
Bolivianische Botschaft
Botschaft Argentinien
Botschaft Bahrain
Botschaft Burkina Faso
Botschaft Burundi
Botschaft Chile
Botschaft Dschibuti
Botschaft Ecuador
Botschaft Gabun
Botschaft Guinea
Botschaft Irland
Botschaft Jamaika
Botschaft Jemen
Botschaft Jordanien
Botschaft Lettlands
Botschaft Peru
Botschaft Polen
Botschaft Sierra Leone
Botschaft Tadschikistan
Botschaft Uruguay
Botschaft der Demokratischen Volksrepublik Korea
Botschaft der Dominikanischen Republik
Botschaft der Elfenbeinküste
Botschaft der Islamischen Republik Afghanistan
Botschaft der Islamischen Republik Iran
Botschaft der Republik Armenien
Botschaft der Republik Aserbaidschan
Botschaft der Republik Belarus
Botschaft der Republik Benin
Botschaft der Republik Costa Rica
Botschaft der Republik Finnland
Botschaft der Republik Ghana
Botschaft der Republik Haiti
Botschaft der Republik Indonesien in Berlin
Botschaft der Republik Irak
Botschaft der Republik Island
Botschaft der Republik Kambodscha
Botschaft der Republik Kamerun
Botschaft der Republik Kap Verde
Botschaft der Republik Kasachstan
Botschaft der Republik Kongo
Botschaft der Republik Korea
Botschaft der Republik Kosovo
Botschaft der Republik Kuba
Botschaft der Republik Litauen
Botschaft der Republik Malawi
Botschaft der Republik Mali
Botschaft der Republik Mazedonien
Botschaft der Republik Moldau
Botschaft der Republik Namibia
Botschaft der Republik Nicaragua
Botschaft der Republik Sambia
Botschaft der Republik Singapur
Botschaft der Republik Sudan
Botschaft der Republik Togo
Botschaft der Republik Tschad
Botschaft der Republik Turkmenistan in der Bundesrepublik Deutschland
Botschaft der Republik Uganda
Botschaft der Republik Usbekistan
Botschaft der Republik Äquatorialguinea
Botschaft der Russischen Föderation
Botschaft der Sozialistischen Republik Vietnam
Botschaft der Syrischen Arabischen Republik
Botschaft der Tschechischen Republik in Berlin
Botschaft der Tunesischen Republik
Botschaft der Ukraine
Botschaft der Union von Myanmar Burma
Botschaft der Vereinigten Arabischen Emirate
Botschaft der Volksrepublik Algerien
Botschaft der Volksrepublik China
Botschaft des Königreiches Dänemark
Botschaft des Königreiches Norwegen
Botschaft des Königreiches Schweden
Botschaft des Königreichs Marokko
Botschaft des Königreichs Saudi-Arabien
Botschaft des Königreichs der Niederlande
Botschaft des Libanon
Botschaft des Staates Eritrea
Botschaft des Staates Israel
Botschaft des Staates Katar?
Botschaft des Sultanats Oman
Botschaft von Bosnien und Herzegowina
Botschaft von Brunei
Botschaft von Bulgarien
Botschaft von El Salvador
Botschaft von Estland
</code></pre></td>
<td><pre><code>
Botschaft von Georgien
Botschaft von Griechenland
Botschaft von Guatemala
Botschaft von Honduras
Botschaft von Irland
Botschaft von Japan
Botschaft von Kenia
Botschaft von Laos
Botschaft von Libyen
Botschaft von Malaysia
Botschaft von Panama
Botschaft von Rumänien
Botschaft von Simbabwe in Berlin
Botschaft von Tansania
Botschaft von Turkmenistan
Botschaft von Ungarn
Botschaft von Zypern
Botschaft Äthiopien
Botschaften von Bahrain, Malta, Monaco und Luxemburg
Botschaften von Portugal
Botschaftsresidenz von Äthiopien
Brasilianische Botschaft
Britische Botschaft
Bulgarische Botschaft
Die Vertretung des Saarlandes beim Bund
Embassy of Pakistan
Embassy of the United States of America
Französische Botschaft
Generaldelegation Palästinas
Griechische Botschaft
Handelsvertretung Hongkong
Hessische Landesvertretung
India Visa Centre
Indische Botschaft
Kanadische Botschaft
Kirgisische Botschaft
Kolumbianische Botschaft
Konsularabteilung der Botschaft der Mongolei
Konsularabteilung der Botschaft der Russischen Föderation
Konsularbüro der Republik Turkmenistan in der Bundesrepublik Deutschland
Konsulat der Republik Ghana
Konsulat der Republik Kolumbien
Konsulat der Seychellen
Kroatische Botschaft
Kulturabteilung der Botschaft der Republik Korea
Königlich Thailändische Botschaft
Landesvertretung Bayern
Landesvertretung Stadt Hamburg
Libanesische Botschafts Residenz
Libysche Botschaft
Liechtensteinische Botschaft
Malteser-Ritterorden
Mauretanische Botschaft in Berlin
Mexikanische Botschaft
New Zealand Embassy Berlin
Republic of South Sudan
Residenz der Republik Jemen
Residenz des Botschafters der Republik Korea
Residenz des Botschafters der Vereinigten Arabischen Emirate
Residenz des Botschafters von Australien
Residenz des Botschafters von Katar
Residenz des Botschafters von Monaco
Residenz des Botschafters von Niger
Residenz des Botschafters von Zypern
Residenz des norwegischen Botschafters
Residenz des türkischen Botschafters
Slowakische Botschaft
Slowenische Botschaft
Spanische Botschaft
Sri Lanka
Taipeh Vertretung
Türkische Botschaft
Vertretung der Europäischen Kommission in Deutschland
Vertretung des Freistaates Sachsen beim Bund
Vertretung des Freistaats Thüringen beim Bund
Vertretung des Landes Baden-Württemberg beim Bund
Vertretung des Landes Brandenburg beim Bund
Vertretung des Landes Mecklenburg-Vorpommern beim Bund
Vertretung des Landes Niedersachsen beim Bund
Vertretung des Landes Nordrhein-Westfalen beim Bund
Vertretung des Landes Rheinland-Pfalz beim Bund
Vertretung des Landes Sachsen-Anhalt beim Bund
Vertretung des Landes Schleswig-Holstein beim Bund
zukünftige griechische Botschaft
Ägyptische Botschaft
Österreichische Botschaft
</code></pre></td>
</tr>
</table>

The data returned was in total 172 items. Some explanations for this are the 'Vertretung' are actually establishments for the German States but are tagged as embassy. Also there is certainly duplicates. For example, 'India Visa Centre' and 'Indische Botschaft'. Another example: 'Botschaft von Griechenland', 'zukünftige griechische Botschaft', 'Griechische Botschaft'.  

I performed the search first only on the nodes_tags and found the U.S embassy was missing the 'anemity = embassy' tag. I found the tag easily searching for Botschaft in nodes_tags. When I included my select in the ways_tags it was also defined in this table, with the English name.

    node_tags
    [(437368383, 52.5157357, 13.3783967, u'Bot45715', 111462, 3, 27912158, u'2015-01-04T15:21:58Z')]
           city | Berlin
         street | Pariser Platz
         suburb | Mitte
        country | DE
       postcode | 10117
      housename | Botschaft der Vereinigten Staaten von Amerika
    housenumber | 2

    ways_tags
    [(195257482, u'overflorian', 125897, u'14', 43350051, u'2016-11-02T12:26:01Z')]
        amenity | embassy
      architect | Architekturbüro Moore Ruble Yudell
       building | embassy
         levels | 4
        country | US
     diplomatic | embassy
         height | 22
           name | Embassy of the United States of America
             de | Botschaft der Vereinigten Staaten von Amerika
             en | Embassy of the United States of America
             fr | Ambassade des États-Unis d'Amérique
           note | The English name is written on the building
     start_date | 2008-05
        website | http://german.germany.usembassy.gov
     wheelchair | yes
      wikipedia | de:Botschaft der Vereinigten Staaten in Berlin



### SIMC tag

There were several tags found with SIMC, such as simc, city:simc or addr:simc. In reviewing these values and researching SIMC, it was found that SIMC is a Poland specific index code for identifying different entities.  
http://dbpedia.org/ontology/simcCode

These nodes were omitted from the data entry.

### Street Abbreviations

In the sample data I found no abbreviations in use. I found this really hard to believe and decided to check the full data set and in fact found Strasse abbreviate to str. in very few occurrences. In a huge dataset, I found this to be pretty impressive.

In a moment of data exploration I chose to append filters to a query to reduce the common street names from view to get an idea of the the different street types in use. I learned 'Am ' and 'An der' could be used as a street type prefix. I also fought there to be many streets that are simply names (in honor of a person or event) without a typical type and these were also valid.

    select distinct value from 
    (select * from nodes_tags union select * from ways_tags) 
    where [key] in ('street', 'addr:street') 
    and value not like '%pfad'  
    and value not like '%straße'  
    and value not like 'am %'  
    and value not like '%damm'  
    and value not like '%allee'  
    and value not like '%weg'  
    and value not like '%platz'  
    and value not like '%ring'  
    and value not like '%steig'  
    and value not like '%assage'  
    

## Data Overview

This section contains information about the data file and data set

    berlin_germany.osm.....2,432,875 KB
    berlin_germany.db......1,339,054 KB
    nodes.csv................845,745 KB
    nodes_tags.csv...........119,961 KB
    ways.csv..................84,955 KB
    ways_nodes.csv...........290,810 KB
    ways_tags.csv............110,212 KB

### Number of Nodes
    SELECT COUNT(id) FROM nodes;

10491003

### Number of Ways
    SELECT COUNT(id) FROM nodes;

1457578

### Number of Users
    SELECT COUNT(DISTINCT(uid)) FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways);

7734

### Most Active Users

    #most active users
    SELECT user, COUNT(uid) FROM (SELECT user, uid FROM nodes UNION ALL SELECT user, uid FROM ways)
    GROUP BY user 
    ORDER BY COUNT(uid) desc 
    LIMIT 20;
    
    #most active days per specific user
    SELECT substr(timestamp,0,11) as workdate, count(user) FROM  
    (SELECT timestamp,user FROM nodes UNION ALL SELECT timestamp,user FROM ways) 
    where user = 'atpl_pilot'  
    group by workdate 
    ORDER BY count(user) desc 
    limit 20 
<table>
<tr>
<td>Most Active Users</td>
<td>atpl_pilot Most Active Days</td>
</tr>
<tr>
<td><pre><code>
atpl_pilot      | 2667549
jacobbraeutigam | 655763
anbr            | 406475
streckenkundler | 378899
r-michael       | 366388
WegefanHB       | 333508
Bot45715        | 275783
toaster         | 181063
Konrad Aust     | 176749
Elwood          | 166591
geozeisig       | 152401
g0ldfish        | 146338
Polarbear       | 131261
Balgofil        | 106715
diverpl         | 106579
Randbewohner    | 103674
wcp79           | 97760
MorbZ           | 95812
OSM-im-nedde    | 90136
glibbertorsten  | 83869
</code></pre></td>
<td><pre><code>
2014-10-23      | 53416
2015-09-11      | 44824
2015-01-28      | 42528
2014-08-28      | 38571
2014-12-26      | 31571
2015-11-16      | 30634
2015-04-23      | 30314
2015-01-30      | 29431
2015-12-08      | 28434
2014-06-28      | 28273
2016-02-02      | 27306
2015-12-07      | 26610
2014-11-05      | 26063
2014-08-29      | 25912
2015-07-23      | 24949
2015-12-13      | 24326
2014-06-24      | 23937
2014-06-27      | 23833
2015-12-23      | 23130
2014-06-25      | 21971
</code></pre></td>
</tr>
</table>

The user atpl_pilot had such significant usage I thought to check his most active days, and found he's been updating Berlin data since 2014. Hats off to you atpl_pilot https://www.openstreetmap.org/user/atpl_pilot

## Additional Details

I used this query to pull the top 20's of some highest frequency tags I found interesting

    SELECT value, COUNT(*) as num 
    FROM (select * from nodes_tags union select * From ways_tags)
    WHERE key='amenity' ## key='tourism', key='shop'
    GROUP BY value
    ORDER BY num DESC
    LIMIT 20;

<table>
<tr>
<td>Top 20 Amenity's</td>
<td>Top 20 Shop Types</td>
<td>Top 20 Tourism Types</td>
</tr>
<tr>
<td><pre><code> 
parking              | 13477
bench                | 9109
restaurant           | 4977
post_box             | 3496
recycling            | 3316
bicycle_parking      | 2922
fast_food            | 2610
kindergarten         | 2580
waste_basket         | 2545
cafe                 | 2366
hunting_stand        | 2033
school               | 1702
place_of_worship     | 1671
telephone            | 1560
vending_machine      | 1517
shelter              | 1213
pub                  | 1180
pharmacy             | 969
toilets              | 950
bank                 | 764
</code></pre></td>
<td><pre><code>   
supermarket     | 1942
bakery          | 1832
hairdresser     | 1488
clothes         | 1344
convenience     | 752
florist         | 741
kiosk           | 646
car_repair      | 566
car             | 536
bicycle         | 397
shoes           | 341
optician        | 322
beverages       | 310
books           | 305
beauty          | 296
chemist         | 265
butcher         | 260
massage         | 254
jewelry         | 246
furniture       | 239
</code></pre></td>
<td><pre><code> 
information     | 3510
picnic_site     | 944
hotel           | 899
artwork         | 789
attraction      | 617
viewpoint       | 429
guest_house     | 334
museum          | 317
chalet          | 191
camp_site       | 169
hostel          | 150
gallery         | 68
zoo             | 60
caravan_site    | 54
theme_park      | 24
motel           | 9
apartment       | 7
yes             | 5
spa_resort      | 3
aquarium        | 2
</code></pre></td>
</tr>
</table>

Taking a deeper look into values for amenity's, we can discover which keys are common for the different value types. This query was used for key='shop' and value='bakery' as well for example, to see the common keys in use for those values.

    SELECT s.key, COUNT(s.id) as num 
    FROM (select * from nodes_tags union select * from ways_tags) m 
    join (select * from nodes_tags union select * from ways_tags) s on m.id = s.id 
    where m.key='amenity' and m.value='restaurant' 
    GROUP BY s.key 
    ORDER BY num DESC 
    LIMIT 10;

An example output:

    amenity         | 4979
    name            | 4812
    wheelchair      | 3417
    cuisine         | 3141
    street          | 2658
    city            | 2568
    postcode        | 2541
    housenumber     | 2527
    country         | 2295
    suburb          | 2212
    
I can see using this query, that about 60%  restaurants have a cuisine tag. I also discovered for the amenity's for fast_food these often use a cuisine tag, whereas neiter of the cafe's used this tag. Cafe's can be tricky to look at in Berlin, as they can either be typical day time coffee/cake places, or open late/overnight cocktail/beer hangouts.

### Top Food Choices
We can get an idea of the preferred food choices in Berlin.

    SELECT s.value, COUNT(s.id) as num 
    FROM (select * from nodes_tags union select * from ways_tags) m 
    join (select * from nodes_tags union select * from ways_tags) s on m.id = s.id 
    where m.key='amenity' and m.value in ('restaurant','fast_food') 
    and s.key in ('cuisine')  
    GROUP BY s.value 
    ORDER BY num DESC
    LIMIT 10;
    
    italian         | 731
    german          | 509
    kebab           | 409
    regional        | 294
    asian           | 293
    burger          | 228
    pizza           | 225
    chinese         | 167
    indian          | 167
    turkish         | 158
    
Italian cuisine seems to be preferred in the city. Pizza, pasta, and the Berlin variant Flammkuchen are quite popular ther. It would certainly be interesting to see the distribution of cuisines throughout suburbs as well.

### Top Religons
Looking at the place_of_worship tag for amenity's I found religion and denomination to be interesting, showing Berlin is predominantly Christian in it's beliefs. 

    SELECT s.value, COUNT(s.id) as num 
    FROM (select * from nodes_tags union select * from ways_tags) m 
    join (select * from nodes_tags union select * from ways_tags) s on m.id = s.id 
    where m.key='amenity' and m.value in ('place_of_worship') 
    and s.key in ('religion')   #denomination
    GROUP BY s.value 
    ORDER BY num DESC 
    LIMIT 10;

<table>
<tr>
<td>By Religion</td>
<td>By Denomination</td>
</tr><tr>
<td><pre><code> 
christian       | 1532
muslim          | 34
jewish          | 9
buddhist        | 7
multifaith      | 4
hindu           | 3
none            | 1
sikh            | 1
</code></pre></td>
<td><pre><code> 
protestant                | 532
lutheran                  | 164
catholic                  | 126
evangelical               | 122
roman_catholic            | 60
baptist                   | 33
new_apostolic             | 28
seventh_day_adventist     | 9
jehovahs_witness          | 8
methodist                 | 8
</code></pre></td>
</tr>
</table>

### Well Fed Suburbs
Which suburbs have the most supermarkets, bakeries, conveniences, butchers and beverages. In Berlin, bakeries are abundant and can often substitute a trip to the grocery store for a little more money. Beverages/Convenience describe what is called 'Spati's', which are typically open very late (or 24 hours), often carrying only drinks, but also many carry typical food stuffs.

    SELECT s.value, COUNT(s.id) as num
    FROM (select * from nodes_tags union select * from ways_tags) m 
    join (select * from nodes_tags union select * from ways_tags) s 
        on m.id = s.id 
    where m.key='shop' and m.value in ('supermarket','bakery','beverages','butcher','convenience') 
        and s.key in ('suburb')  
    GROUP BY s.value 
    ORDER BY num DESC 
    limit 10 

<table>
<tr>
<td>With Convenience</td>
<td>Without Convenience</td>
</tr><tr>
<td><pre><code> 
Friedrichshain            | 96
Kreuzberg                 | 89
Charlottenburg            | 88
Prenzlauer Berg           | 81
Schöneberg                | 75
Mitte                     | 70
Lichterfelde              | 52
Neukölln                  | 47
Gesundbrunnen             | 40
Wilmersdorf               | 40
</code></pre></td>
<td><pre><code> 
Charlottenburg            | 78
Friedrichshain            | 72
Schöneberg                | 68
Kreuzberg                 | 65
Mitte                     | 64
Prenzlauer Berg           | 62
Lichterfelde              | 49
Neukölln                  | 38
Wilmersdorf               | 37
Gesundbrunnen             | 32
</code></pre></td>
</tr>
</table>

I found it interesting to compare these with/without including conveniecne (Spati) shops. We can perhaps correlate the difference for Friedrichshain (+24), Kreuzberg (+24), for example for being popular nightlife areas.


### Athletics
We can look into the liesure tag for information on what sports are most prevelant

    SELECT s.value, COUNT(s.id) as num 
    FROM (select * from nodes_tags union select * from ways_tags) m 
    join (select * from nodes_tags union select * from ways_tags) s 
        on m.id = s.id 
    where m.key='leisure' and m.value in ('pitch','sports_centre') 
        and s.key in ('sport')  
    GROUP BY s.value 
    ORDER BY num DESC 
    limit 10 
    
    soccer                    | 1532
    tennis                    | 843
    multi                     | 797
    table_tennis              | 451
    basketball                | 269
    golf                      | 232
    beachvolleyball           | 206
    equestrian                | 161
    volleyball                | 94
    athletics                 | 86
    
Not really surprising since Soccer is hugely popular in Berlin and in general, Germany.



## Conclusion

Overall I found openstreetmap users to have put forth a good effort in keeping Berlin up to date. It is known as a city that is constantly in flux, so maintaining accuracy would require constant vigilance. 

My biggest issue with the data was discovering that some postal codes had inconsistent suburb tag. I felt the suburb tag can yield interesting correlations, especially since postal codes could be shared with different suburbs within a Bezirk (an overarching suburb). 

One way to correct this would be to develop a script to scrape postal data from wikipedia. Each of the 12 Bezirks can be found in: https://de.wikipedia.org/wiki/Berliner_Bezirke#Nach_der_Wiedervereinigung. Each Bezirk has an Ortsteile section (for example) https://de.wikipedia.org/wiki/Bezirk_Neuk%C3%B6lln#Ortsteile's that contains specific info about the suburb. This includes on the right, postal data (https://de.wikipedia.org/wiki/Berlin-Neuk%C3%B6lln) in tabled data. We could use this keep the suburb information accurate. One issue I can see would be in choosing between 2 suburbs for a specific postal code. This dataframe could also be used to update node tags that are missing a suburb tag, but have a postcode tag.

Another benefit could be using the Google Maps Geocoding API. With the API, one can reverse geocode latitude and longitude coordinates. An example of this process may be as follows
1. A user may use video mapping for a certain section of the city while enabling a GPS navigation record. Galileo is a useful app that allows you to record your travel, and export the file to GPX or KML.
2. Once complete, the user may use the GPX extract as input to reverse geocode locations and create a data set of addresses and other useful information for approximate points.
3. The user could add tags for missing info, and validate the info (use video as another form of validation). 
4. If the timings in the video could be synced with the points of the GPX entries this could make the validation a more streamlined process (for example, at 2:42 at approx. coordinates '52.5157357,13.3783967' there is a valid google maps address returned)

Some drawbacks with this approach are the API data could return multiple results for each coordinate. For the above coordinates in point 4, 9 'address_components' results are returned. This data could be merged into one set to reduce duplicates, or flagged if the addresses are different.


#### Google maps reverse geocode for 52.5157357,13.3783967
    {
       "results" : [
          {
             "address_components" : [
                {
                   "long_name" : "2",
                   "short_name" : "2",
                   "types" : [ "street_number" ]
                },
                {
                   "long_name" : "Pariser Platz",
                   "short_name" : "Pariser Platz",
                   "types" : [ "route" ]
                },
                {
                   "long_name" : "Mitte",
                   "short_name" : "Mitte",
                   "types" : [ "political", "sublocality", "sublocality_level_1" ]
                },
                {
                   "long_name" : "Berlin",
                   "short_name" : "Berlin",
                   "types" : [ "locality", "political" ]
                },
                {
                   "long_name" : "Berlin",
                   "short_name" : "Berlin",
                   "types" : [ "administrative_area_level_1", "political" ]
                },
                {
                   "long_name" : "Germany",
                   "short_name" : "DE",
                   "types" : [ "country", "political" ]
                },
                {
                   "long_name" : "10117",
                   "short_name" : "10117",
                   "types" : [ "postal_code" ]
                }
             ],
             "formatted_address" : "Pariser Platz 2, 10117 Berlin, Germany",
             "geometry" : {
                "bounds" : {
                   "northeast" : {
                      "lat" : 52.51580209999999,
                      "lng" : 13.3788361
                   },
                   "southwest" : {
                      "lat" : 52.5148074,
                      "lng" : 13.3779006
                   }
                },
                "location" : {
                   "lat" : 52.51530469999999,
                   "lng" : 13.3783684
                },
                "location_type" : "ROOFTOP",
                "viewport" : {
                   "northeast" : {
                      "lat" : 52.51665373029149,
                      "lng" : 13.3797173302915
                   },
                   "southwest" : {
                      "lat" : 52.5139557697085,
                      "lng" : 13.3770193697085
                   }
                }
             },
             "place_id" : "ChIJP889RMZRqEcRSHnCrvbtXew",
             "types" : [ "premise" ]
          },
          {
             "address_components" : [
                {
                   "long_name" : "2",
                   "short_name" : "2",
                   "types" : [ "street_number" ]
                },
                {
                   "long_name" : "Brandenburger Tor",
                   "short_name" : "Brandenburger Tor",
                   "types" : [ "route" ]
                },
                {
                   "long_name" : "Mitte",
                   "short_name" : "Mitte",
                   "types" : [ "political", "sublocality", "sublocality_level_1" ]
                },
                {
                   "long_name" : "Berlin",
                   "short_name" : "Berlin",
                   "types" : [ "locality", "political" ]
                },
                {
                   "long_name" : "Berlin",
                   "short_name" : "Berlin",
                   "types" : [ "administrative_area_level_1", "political" ]
                },
                {
                   "long_name" : "Germany",
                   "short_name" : "DE",
                   "types" : [ "country", "political" ]
                },
                {
                   "long_name" : "10117",
                   "short_name" : "10117",
                   "types" : [ "postal_code" ]
                }
             ],
             "formatted_address" : "Brandenburger Tor 2, 10117 Berlin, Germany",
             "geometry" : {
                "location" : {
                   "lat" : 52.5160505,
                   "lng" : 13.3780092
                },
                "location_type" : "ROOFTOP",
                "viewport" : {
                   "northeast" : {
                      "lat" : 52.51739948029149,
                      "lng" : 13.3793581802915
                   },
                   "southwest" : {
                      "lat" : 52.5147015197085,
                      "lng" : 13.3766602197085
                   }
                }
             },
             "place_id" : "ChIJQe35VsZRqEcR-QSHa2q1njg",
             "types" : [ "street_address" ]
          },
          {
             "address_components" : [
                {
                   "long_name" : "Mitte",
                   "short_name" : "Mitte",
                   "types" : [ "political", "sublocality", "sublocality_level_2" ]
                },
                {
                   "long_name" : "Mitte",
                   "short_name" : "Mitte",
                   "types" : [ "political", "sublocality", "sublocality_level_1" ]
                },
                {
                   "long_name" : "Berlin",
                   "short_name" : "Berlin",
                   "types" : [ "locality", "political" ]
                },
                {
                   "long_name" : "Berlin",
                   "short_name" : "Berlin",
                   "types" : [ "administrative_area_level_1", "political" ]
                },
                {
                   "long_name" : "Germany",
                   "short_name" : "DE",
                   "types" : [ "country", "political" ]
                }
             ],
             "formatted_address" : "Mitte, Berlin, Germany",
             "geometry" : {
                "bounds" : {
                   "northeast" : {
                      "lat" : 52.5403962,
                      "lng" : 13.4293586
                   },
                   "southwest" : {
                      "lat" : 52.5040199,
                      "lng" : 13.3658543
                   }
                },
                "location" : {
                   "lat" : 52.519444,
                   "lng" : 13.406667
                },
                "location_type" : "APPROXIMATE",
                "viewport" : {
                   "northeast" : {
                      "lat" : 52.5403962,
                      "lng" : 13.4293586
                   },
                   "southwest" : {
                      "lat" : 52.5040199,
                      "lng" : 13.3658543
                   }
                }
             },
             "place_id" : "ChIJjw3Y6t9RqEcR8jUVWEcgISY",
             "types" : [ "neighborhood", "political", "sublocality", "sublocality_level_2" ]
          },
          {
             "address_components" : [
                {
                   "long_name" : "Mitte",
                   "short_name" : "Mitte",
                   "types" : [ "political", "sublocality", "sublocality_level_1" ]
                },
                {
                   "long_name" : "Berlin",
                   "short_name" : "Berlin",
                   "types" : [ "locality", "political" ]
                },
                {
                   "long_name" : "Berlin",
                   "short_name" : "Berlin",
                   "types" : [ "administrative_area_level_1", "political" ]
                },
                {
                   "long_name" : "Germany",
                   "short_name" : "DE",
                   "types" : [ "country", "political" ]
                }
             ],
             "formatted_address" : "Mitte, Berlin, Germany",
             "geometry" : {
                "bounds" : {
                   "northeast" : {
                      "lat" : 52.5677268,
                      "lng" : 13.4293586
                   },
                   "southwest" : {
                      "lat" : 52.4987314,
                      "lng" : 13.3015252
                   }
                },
                "location" : {
                   "lat" : 52.5306438,
                   "lng" : 13.3830683
                },
                "location_type" : "APPROXIMATE",
                "viewport" : {
                   "northeast" : {
                      "lat" : 52.5677268,
                      "lng" : 13.4293586
                   },
                   "southwest" : {
                      "lat" : 52.4987314,
                      "lng" : 13.3015252
                   }
                }
             },
             "place_id" : "ChIJAUK8it1RqEcRwKtfW0YgIQU",
             "types" : [ "political", "sublocality", "sublocality_level_1" ]
          },
          {
             "address_components" : [
                {
                   "long_name" : "Berlin",
                   "short_name" : "Berlin",
                   "types" : [ "locality", "political" ]
                },
                {
                   "long_name" : "Berlin",
                   "short_name" : "Berlin",
                   "types" : [ "administrative_area_level_1", "political" ]
                },
                {
                   "long_name" : "Germany",
                   "short_name" : "DE",
                   "types" : [ "country", "political" ]
                }
             ],
             "formatted_address" : "Berlin, Germany",
             "geometry" : {
                "bounds" : {
                   "northeast" : {
                      "lat" : 52.6754542,
                      "lng" : 13.7611176
                   },
                   "southwest" : {
                      "lat" : 52.338234,
                      "lng" : 13.088346
                   }
                },
                "location" : {
                   "lat" : 52.52000659999999,
                   "lng" : 13.404954
                },
                "location_type" : "APPROXIMATE",
                "viewport" : {
                   "northeast" : {
                      "lat" : 52.6754542,
                      "lng" : 13.7611175
                   },
                   "southwest" : {
                      "lat" : 52.33962959999999,
                      "lng" : 13.0911719
                   }
                }
             },
             "place_id" : "ChIJAVkDPzdOqEcRcDteW0YgIQQ",
             "types" : [ "locality", "political" ]
          },
          {
             "address_components" : [
                {
                   "long_name" : "10117",
                   "short_name" : "10117",
                   "types" : [ "postal_code" ]
                },
                {
                   "long_name" : "Mitte",
                   "short_name" : "Mitte",
                   "types" : [ "political", "sublocality", "sublocality_level_1" ]
                },
                {
                   "long_name" : "Berlin",
                   "short_name" : "Berlin",
                   "types" : [ "locality", "political" ]
                },
                {
                   "long_name" : "Berlin",
                   "short_name" : "Berlin",
                   "types" : [ "administrative_area_level_1", "political" ]
                },
                {
                   "long_name" : "Germany",
                   "short_name" : "DE",
                   "types" : [ "country", "political" ]
                }
             ],
             "formatted_address" : "10117 Berlin, Germany",
             "geometry" : {
                "bounds" : {
                   "northeast" : {
                      "lat" : 52.528411,
                      "lng" : 13.405512
                   },
                   "southwest" : {
                      "lat" : 52.50703009999999,
                      "lng" : 13.372667
                   }
                },
                "location" : {
                   "lat" : 52.5155098,
                   "lng" : 13.3847539
                },
                "location_type" : "APPROXIMATE",
                "viewport" : {
                   "northeast" : {
                      "lat" : 52.528411,
                      "lng" : 13.405512
                   },
                   "southwest" : {
                      "lat" : 52.50703009999999,
                      "lng" : 13.372667
                   }
                }
             },
             "place_id" : "ChIJ-QcxXsNRqEcRUPc_lUkgIRw",
             "types" : [ "postal_code" ]
          },
          {
             "address_components" : [
                {
                   "long_name" : "Berlin",
                   "short_name" : "Berlin",
                   "types" : [ "administrative_area_level_1", "political" ]
                },
                {
                   "long_name" : "Germany",
                   "short_name" : "DE",
                   "types" : [ "country", "political" ]
                }
             ],
             "formatted_address" : "Berlin, Germany",
             "geometry" : {
                "bounds" : {
                   "northeast" : {
                      "lat" : 52.6754542,
                      "lng" : 13.7611175
                   },
                   "southwest" : {
                      "lat" : 52.338234,
                      "lng" : 13.088346
                   }
                },
                "location" : {
                   "lat" : 52.4938053,
                   "lng" : 13.4552919
                },
                "location_type" : "APPROXIMATE",
                "viewport" : {
                   "northeast" : {
                      "lat" : 52.6754542,
                      "lng" : 13.7611175
                   },
                   "southwest" : {
                      "lat" : 52.33962959999999,
                      "lng" : 13.0911719
                   }
                }
             },
             "place_id" : "ChIJ8_KccStOqEcRhtFXjKWPuo0",
             "types" : [
                "administrative_area_level_1",
                "establishment",
                "point_of_interest",
                "political"
             ]
          },
          {
             "address_components" : [
                {
                   "long_name" : "Berlin Metropolitan Area",
                   "short_name" : "Berlin Metropolitan Area",
                   "types" : [ "political" ]
                },
                {
                   "long_name" : "Germany",
                   "short_name" : "DE",
                   "types" : [ "country", "political" ]
                }
             ],
             "formatted_address" : "Berlin Metropolitan Area, Germany",
             "geometry" : {
                "bounds" : {
                   "northeast" : {
                      "lat" : 53.55898,
                      "lng" : 14.7658261
                   },
                   "southwest" : {
                      "lat" : 51.3590586,
                      "lng" : 11.265727
                   }
                },
                "location" : {
                   "lat" : 52.268409,
                   "lng" : 13.5287229
                },
                "location_type" : "APPROXIMATE",
                "viewport" : {
                   "northeast" : {
                      "lat" : 53.55898,
                      "lng" : 14.765826
                   },
                   "southwest" : {
                      "lat" : 51.3590586,
                      "lng" : 11.265727
                   }
                }
             },
             "place_id" : "ChIJZ4kamin3qEcRQ5VPQ7O8dWY",
             "types" : [ "political" ]
          },
          {
             "address_components" : [
                {
                   "long_name" : "Germany",
                   "short_name" : "DE",
                   "types" : [ "country", "political" ]
                }
             ],
             "formatted_address" : "Germany",
             "geometry" : {
                "bounds" : {
                   "northeast" : {
                      "lat" : 55.0815,
                      "lng" : 15.0418962
                   },
                   "southwest" : {
                      "lat" : 47.2701115,
                      "lng" : 5.8663425
                   }
                },
                "location" : {
                   "lat" : 51.165691,
                   "lng" : 10.451526
                },
                "location_type" : "APPROXIMATE",
                "viewport" : {
                   "northeast" : {
                      "lat" : 55.05812359999999,
                      "lng" : 15.0417724
                   },
                   "southwest" : {
                      "lat" : 47.2702482,
                      "lng" : 5.8664874
                   }
                }
             },
             "place_id" : "ChIJa76xwh5ymkcRW-WRjmtd6HU",
             "types" : [ "country", "political" ]
          }
       ],
       "status" : "OK"
    }